In [195]:
#Libraries to Import:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


plt.rcParams['figure.figsize'] = [15, 6]

import warnings
warnings.filterwarnings("ignore")


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from mlxtend.feature_selection import SequentialFeatureSelector
import statsmodels.api as sma
import statsmodels.stats.api as ssa
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson
from sklearn.model_selection import GridSearchCV

In [161]:
df = pd.read_excel('Admission.xlsx')
df.head()


,SlNo,Gender,Percent_SSC,Board_SSC,Percent_HSC,Board_HSC,Stream_HSC,Percent_Degree,Course_Degree,Experience_Yrs,Entrance_Test,Percentile_ET,Percent_MBA,Specialization_MBA,Marks_Communication,Marks_Projectwork,Marks_BOCA,Placement,Salary
0,1,M,62.00,Others,88.00,Others,Commerce,52.00,Science,0,MAT,55.0,58.80,Marketing_HR,50,65,74,Placed,270000.0
1,2,M,76.33,ICSE,75.33,Others,Science,75.48,Computer_Applications,1,MAT,86.5,66.28,Marketing_Finance,69,70,75,Placed,200000.0
2,3,M,72.00,Others,78.00,Others,Commerce,66.63,Engineering,0,NaN,0.0,52.91,Marketing_Finance,50,61,59,Not_Placed,NaN
3,4,M,60.00,CBSE,63.00,CBSE,Arts,58.00,Management,0,MAT,75.0,57.80,Marketing_Finance,54,66,62,Placed,250000.0
4,5,M,61.00,CBSE,55.00,ISC,Science,54.00,Engineering,1,MAT,66.0,59.43,Marketing_HR,52,65,67,Placed,180000.0


In [162]:
df.isnull().sum()*100/len(df)

SlNo                    0.000000
Gender                  0.000000
Percent_SSC             0.000000
Board_SSC               0.000000
Percent_HSC             0.000000
Board_HSC               0.000000
Stream_HSC              0.000000
Percent_Degree          0.000000
Course_Degree           0.000000
Experience_Yrs          0.000000
Entrance_Test          17.135550
Percentile_ET           0.000000
Percent_MBA             0.000000
Specialization_MBA      0.000000
Marks_Communication     0.000000
Marks_Projectwork       0.000000
Marks_BOCA              0.000000
Placement               0.000000
Salary                  3.580563
dtype: float64

In [163]:
df['Entrance_Test'] = df['Entrance_Test'].fillna('Management Seat')

In [164]:
df[df['Entrance_Test'] == 'Management Seat'][['Percentile_ET']].describe()

,Percentile_ET
count,67.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


#### Checking Salary

In [165]:
df[df['Placement'] == "Not_Placed"][['Salary']].isnull().sum()

Salary    4
dtype: int64

In [166]:
df[df['Placement'] == "Placed"][['Salary']].isnull().sum()

Salary    10
dtype: int64

In [167]:
df['Salary'].isnull().sum()

14

In [168]:
df['Salary'] = df['Salary'].fillna(df['Salary'].median())
#imputing with median, because even if data is skewed median wil not effect data much

In [169]:
df_num = df.select_dtypes(include = np.number)
df_cat = df.select_dtypes(include = 'object')

In [170]:
df_num.columns

Index(['SlNo', 'Percent_SSC', 'Percent_HSC', 'Percent_Degree',
       'Experience_Yrs', 'Percentile_ET', 'Percent_MBA', 'Marks_Communication',
       'Marks_Projectwork', 'Marks_BOCA', 'Salary'],
      dtype='object')

In [171]:
### Apply get dummies on CAT data

In [172]:
dummy_data = pd.get_dummies(df_cat, drop_first=True)

In [173]:
df_final = pd.concat([df_num, dummy_data], axis = 1)

In [174]:
df_final.drop(['SlNo','Placement_Placed'], axis = 1, inplace = True)

In [175]:
df_final

,Percent_SSC,Percent_HSC,Percent_Degree,Experience_Yrs,Percentile_ET,Percent_MBA,Marks_Communication,Marks_Projectwork,Marks_BOCA,Salary,...,Entrance_Test_GCET,Entrance_Test_GMAT,Entrance_Test_GSAT,Entrance_Test_KMAT,Entrance_Test_MAT,Entrance_Test_Management Seat,Entrance_Test_PGCET,Entrance_Test_XAT,Specialization_MBA_Marketing_HR,Specialization_MBA_Marketing_IB
0,62.00,88.00,52.00,0,55.00,58.80,50,65,74,270000.0,...,0,0,0,0,1,0,0,0,1,0
1,76.33,75.33,75.48,1,86.50,66.28,69,70,75,200000.0,...,0,0,0,0,1,0,0,0,0,0
2,72.00,78.00,66.63,0,0.00,52.91,50,61,59,250000.0,...,0,0,0,0,0,1,0,0,0,0
3,60.00,63.00,58.00,0,75.00,57.80,54,66,62,250000.0,...,0,0,0,0,1,0,0,0,0,0
4,61.00,55.00,54.00,1,66.00,59.43,52,65,67,180000.0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,62.60,50.60,52.00,0,34.53,58.63,62,57,56,162000.0,...,0,0,0,0,1,0,0,0,1,0
387,77.00,49.50,64.00,0,50.53,59.50,70,59,56,450000.0,...,0,0,0,0,1,0,0,0,0,0
388,66.40,72.00,57.00,0,60.00,61.63,73,73,54,240000.0,...,0,0,0,0,1,0,0,0,0,1
389,70.00,77.00,53.00,0,77.00,70.17,67,77,57,300000.0,...,0,0,0,0,1,0,0,0,0,0


In [176]:
X = df_final.drop('Salary', axis = 1)
y = df_final['Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [177]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(273, 32)
(118, 32)
(273,)
(118,)


- Import the model library
- Initiate
- Fit
- Predict
- Metric Evaluation

### Linear Regression

In [178]:
lr = LinearRegression()
lr_base_model = lr.fit(X_train, y_train)

In [179]:
lr_base_model.intercept_

-271340.07260046515

In [180]:
lr_base_model.coef_

array([ 2.89373325e+03, -1.80831309e+02,  1.24176805e+02,  2.95662486e+04,
        7.66410481e+02,  1.69055262e+03, -1.48224026e+03,  2.92377423e+03,
        1.96223250e+02,  6.80012453e+04, -3.30768474e+04, -4.33124876e+04,
        6.74030750e+03,  1.27367646e+04,  1.32302180e+04,  6.12355669e+03,
        3.33307655e+04, -1.11122346e+04,  1.19898058e+04,  1.64606652e+04,
       -9.38786796e+03,  2.34783808e+04,  7.55905541e+04,  2.18278728e-11,
        8.04150841e+04,  1.53297823e+04, -3.70969395e+04,  1.51580920e+04,
       -2.64808613e+04,  3.55282872e+03, -5.04053952e+03, -4.18449575e+04])

In [181]:
y_pred_BM = lr_base_model.predict(X_test)

#### R2 score and MSE

In [182]:
# syntax - r2(actual, predicted)

In [183]:
r2_score(y_test, y_pred_BM)

-0.018079363921897462

In [184]:
np.sqrt(mean_squared_error(y_test, y_pred_BM))

136080.90335109408

In [185]:
# root itself high da dei....

In [186]:
## justc checking with xtrain itself 

In [187]:
train_pred = lr_base_model.predict(X_train)

In [188]:
r2_score(y_train, train_pred)

0.20546134529072457

In [189]:
df.corr()

,SlNo,Percent_SSC,Percent_HSC,Percent_Degree,Experience_Yrs,Percentile_ET,Percent_MBA,Marks_Communication,Marks_Projectwork,Marks_BOCA,Salary
SlNo,1.000000,0.003387,-0.069011,0.039278,0.015916,0.037508,0.076063,0.137206,0.018849,0.039268,-0.101662
Percent_SSC,0.003387,1.000000,0.396585,0.413072,-0.015237,0.211517,0.475638,0.476279,0.132496,0.271597,0.196996
Percent_HSC,-0.069011,0.396585,1.000000,0.338943,-0.042638,0.151457,0.380495,0.321432,0.160446,0.156589,0.090740
Percent_Degree,0.039278,0.413072,0.338943,1.000000,-0.029147,0.213127,0.447138,0.412716,0.191756,0.268876,0.090828
Experience_Yrs,0.015916,-0.015237,-0.042638,-0.029147,1.000000,-0.009219,0.160725,0.086718,0.142599,0.172957,0.154696
Percentile_ET,0.037508,0.211517,0.151457,0.213127,-0.009219,1.000000,0.214161,0.200447,0.146226,0.138224,0.149208
Percent_MBA,0.076063,0.475638,0.380495,0.447138,0.160725,0.214161,1.000000,0.706999,0.435558,0.476736,0.169224
Marks_Communication,0.137206,0.476279,0.321432,0.412716,0.086718,0.200447,0.706999,1.000000,0.308851,0.210567,0.123384
Marks_Projectwork,0.018849,0.132496,0.160446,0.191756,0.142599,0.146226,0.435558,0.308851,1.000000,0.260201,0.145916
Marks_BOCA,0.039268,0.271597,0.156589,0.268876,0.172957,0.138224,0.476736,0.210567,0.260201,1.000000,0.125820


### Lasso

In [190]:
lasso = Lasso()
lasso_BM = lasso.fit(X_train, y_train)
lasso_pred_BM = lasso_BM.predict(X_test)
print('RMSE lasso_BM', np.sqrt(mean_squared_error(y_test, lasso_pred_BM)))
print('r_square lasso_bm', r2_score(y_test, lasso_pred_BM))

RMSE lasso_BM 136075.6545892024
r_square lasso_bm -0.018000828977145167


### Ridge

In [191]:
ridge = Ridge()
ridge_BM = ridge.fit(X_train, y_train)
ridge_pred_BM = ridge_BM.predict(X_test)
print('RMSE lasso_BM', np.sqrt(mean_squared_error(y_test, ridge_pred_BM)))
print('r_square lasso_bm', r2_score(y_test, ridge_pred_BM))

RMSE lasso_BM 135542.2607586074
r_square lasso_bm -0.010035684325727878


### Elastic Net

In [194]:
enet = ElasticNet()
enet_BM = enet.fit(X_train, y_train)
enet_BM_pred = enet_BM.predict(X_test)
print('RMSE enet', np.sqrt(mean_squared_error(y_test, enet_BM_pred)))
print('r_square enet', r2_score(y_test, enet_BM_pred))

RMSE enet 133915.77309055798
r_square enet 0.014059440399710965


### SGD (Stochastic Gradient Descent)

In [244]:
SGD = SGDRegressor()
SGD_BM = SGD.fit(X_train, y_train)
SGD_BM_pred = SGD_BM.predict(X_test)
print('RMSE SGD', np.sqrt(mean_squared_error(y_test, SGD_BM_pred)))
print('r_square SGD', r2_score(y_test, SGD_BM_pred))

RMSE SGD 2016945068905.1912
r_square SGD -223653350301185.9


## Grid Search CV

In [245]:
tuned_param = [{'alpha':[0.1,0.5,0.7], 'l1_ratio':[0.3,0.6,0.9]}]
enet = ElasticNet()

enet_grid = GridSearchCV(estimator=enet, param_grid=tuned_param, cv = 3)
enet_grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=ElasticNet(),
             param_grid=[{'alpha': [0.1, 0.5, 0.7],
                          'l1_ratio': [0.3, 0.6, 0.9]}])

In [246]:
enet_grid.best_params_

{'alpha': 0.5, 'l1_ratio': 0.6}